# 1차 전처리 과정을 끝낸 데이터들을 불러오는 과정

## 모델학습용 데이터 병합

In [1]:
import pandas as pd
import numpy as np

In [2]:
hospital = pd.read_csv('/home/jupyter/climate/data/back_hospital.csv', header = 0)
sfc_prsr = pd.read_csv('/home/jupyter/climate/data/final_sfc_prsr_0723.csv', header = 0)
sfc_td = pd.read_csv('/home/jupyter/climate/data/final_sfc_td_0723.csv', header = 0)
sfc_ta = pd.read_csv('/home/jupyter/climate/data/final_sfc_ta_0723.csv', header = 0)
ydst = pd.read_csv('/home/jupyter/climate/data/final_ydst_pm10_ver1_0723.csv', header = 0)
aws_ta = pd.read_csv('/home/jupyter/climate/data/final_aws_ta_0713.csv', header = 0)
fore_rhm = pd.read_csv('/home/jupyter/climate/data/final_fore_rhm_0801_2313.csv', header = 0)

### 병합의 용이성을 위해 변수명을 바꿔주는 과정

In [63]:
print(hospital.columns)
print(sfc_prsr.columns)
print(sfc_td.columns)
print(sfc_ta.columns)
print(ydst.columns)
print(aws_ta.columns)
print(fore_rhm.columns)

Index(['Unnamed: 0', 'back_hospital.yyyymmdd', 'back_hospital.area',
       'back_hospital.sex', 'back_hospital.frequency'],
      dtype='object')
Index(['yyyymmdd', 'tma', 'stn_id', 'avg_ps', 'max_ps', 'min_ps', 'name',
       'address', 'lat', 'long', 'add'],
      dtype='object')
Index(['yyyymmdd', 'tma', 'stn_id', 'avg_td', 'max_td', 'max_td_hrmt',
       'min_td', 'min_td_hrmt', 'name', 'address', 'lat', 'long', 'add'],
      dtype='object')
Index(['yyyymmdd', 'tma', 'stn_id', 'avg_ta', 'max_ta', 'min_ta', 'name',
       'address', 'lat', 'long', 'add'],
      dtype='object')
Index(['yyyymmdd', 'area', 'so2', 'co', 'o3', 'no2', 'pm10', 'isnull'], dtype='object')
Index(['Unnamed: 0', 'yyyymmdd', 'tma', 'stn_id', 'avg_ta', 'max_ta', 'min_ta',
       'min_mrng_ta', 'max_dytm_ta', 'min_nght_ta', 'name', 'address', 'lat',
       'long', 'add'],
      dtype='object')
Index(['yyyymmdd', 'tma', 'add', 'address', 'value', 'stn_id', 'long', 'lat'], dtype='object')


In [3]:
hospital = hospital.rename(columns={'back_hospital.area' : 'add'})
hospital = hospital.rename(columns={'back_hospital.yyyymmdd' : 'yyyymmdd'})
hospital = hospital.rename(columns={'back_hospital.sex' : 'sex'})
hospital = hospital.rename(columns={'back_hospital.frequency' : 'frequency'})
ydst = ydst.rename(columns={'area' : 'add'})

### 시도별로 빠진 곳이 있는지 확인하기

In [4]:
print(hospital['add'].unique())
print(sfc_prsr['add'].unique())
print(sfc_td['add'].unique())
print(sfc_ta['add'].unique())
print(ydst['add'].unique())
print(aws_ta['add'].unique())
print(fore_rhm['add'].unique())

['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
['강원' '경기' '인천' '충남' '경북' '충북' '울산' '경남' '제주' '전북' '광주' '전남' '부산' '대전'
 '대구' '서울']
['강원' '경기' '충북' '경북' '전북' '광주' '경남' '전남' '제주' '충남' '대구' '울산' '인천' '서울'
 '부산' '대전']
['강원' '경기' '인천' '충북' '전북' '전남' '제주' '충남' '경남' '경북' '서울' '울산' '대구' '부산'
 '대전' '광주']
['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
['전북' '경남' '전남' '강원' '충북' '경북' '서울' '경기' '인천' '충남' '제주' '울산' '부산' '광주'
 '대전' '대구' '세종']
['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']


### 파일명에 sfc가 들어있는 데이터들은 세종시 데이터가 없기에 세종시를 둘러싼 각 데이터의 대전, 충남, 충북 평균으로 보간한다.  
### 충남, 충북, 대전에 속한 관측소들의 날짜별 평균을 구하여 각 데이터프레임의 행에 추가하는 코드이다.

In [5]:
prsr_sejong = sfc_prsr[sfc_prsr['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_prsr = sfc_prsr.append(prsr_sejong)

ta_sejong = sfc_ta[sfc_ta['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_ta = sfc_ta.append(ta_sejong)

td_sejong = sfc_td[sfc_td['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_td = sfc_td.append(td_sejong)

### 각 변수에 따라 시도별 데이터가 필요하기 때문에 각 데이터프레임에서 날짜와 시도가 동일한 행들끼리 평균을 산출하였다. 

In [6]:
sfc_prsr_grouped = sfc_prsr.groupby(['yyyymmdd','add']).mean()
sfc_ta_grouped = sfc_ta.groupby(['yyyymmdd','add']).mean()
sfc_td_grouped = sfc_td.groupby(['yyyymmdd','add']).mean()
ydst_grouped = ydst.groupby(['yyyymmdd','add']).mean()
aws_ta_grouped = aws_ta.groupby(['yyyymmdd','add']).mean()
fore_rhm_grouped = fore_rhm.groupby(['yyyymmdd','add']).mean()

### 모든 데이터프레임이 동일한 행수를 가지고 있는 것을 확인하고, 모든 데이터프레임을 날짜와 시도에 따라 병합하는 과정을 진행한다. 

In [7]:
merged_1 = pd.merge(hospital, sfc_prsr_grouped, on = ['add','yyyymmdd'])
merged_2 = pd.merge(merged_1, sfc_ta_grouped, on = ['add','yyyymmdd'])
merged_3 = pd.merge(merged_2, sfc_ta_grouped, on = ['add','yyyymmdd'])
merged_4 = pd.merge(merged_3, sfc_td_grouped, on = ['add','yyyymmdd'])
merged_5 = pd.merge(merged_4, ydst_grouped, on = ['add','yyyymmdd'])
merged_6 = pd.merge(merged_5, aws_ta_grouped, on = ['add','yyyymmdd'])
merged_7 = pd.merge(merged_6, fore_rhm_grouped, on = ['add','yyyymmdd'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'lat_x', 'long_x', 'stn_id_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'lat_x', 'long_x', 'stn_id_x'} in the result is deprecated and will raise a MergeError in a future version.
  import sys


### 데이터 병합과정에서 불필요하게 추가된 변수들 및 사용하지 않기로 한 변수들을 제거해주는 코드

In [8]:
# 모든 변수명 확인
merged_7.columns

Index(['Unnamed: 0_x', 'yyyymmdd', 'add', 'sex', 'frequency', 'stn_id_x',
       'avg_ps', 'max_ps', 'min_ps', 'lat_x', 'long_x', 'stn_id_y', 'avg_ta_x',
       'max_ta_x', 'min_ta_x', 'lat_y', 'long_y', 'stn_id_x', 'avg_ta_y',
       'max_ta_y', 'min_ta_y', 'lat_x', 'long_x', 'stn_id_y', 'avg_td',
       'max_td', 'max_td_hrmt', 'min_td', 'min_td_hrmt', 'lat_y', 'long_y',
       'so2', 'co', 'o3', 'no2', 'pm10', 'isnull', 'Unnamed: 0_y', 'stn_id_x',
       'avg_ta', 'max_ta', 'min_ta', 'min_mrng_ta', 'max_dytm_ta',
       'min_nght_ta', 'lat_x', 'long_x', 'value', 'stn_id_y', 'long_y',
       'lat_y'],
      dtype='object')

In [9]:
merged_7.drop(['Unnamed: 0_x', 'stn_id_x', 'lat_x', 'long_x', 'isnull', 'stn_id_y', 'lat_y', 'long_y', 'stn_id_x', 'Unnamed: 0_y', 'min_mrng_ta', 'max_dytm_ta', 'min_nght_ta'], axis=1, inplace=True)

### 데이터 확인 및 저장

In [71]:
merged_7

,yyyymmdd,add,sex,frequency,avg_ps,max_ps,min_ps,avg_ta_x,max_ta_x,min_ta_x,...,o3,no2,pm10,avg_ta,max_ta,min_ta,min_mrng_ta,max_dytm_ta,min_nght_ta,value
0,20120101,강원,1,3,1023.085714,1025.485714,1019.465714,-2.690000,1.482857,-6.760000,...,0.019098,0.020873,84.337500,-3.804615,0.408308,-8.395385,-5.596308,0.380615,-10.599385,62.357641
1,20120101,강원,2,3,1023.085714,1025.485714,1019.465714,-2.690000,1.482857,-6.760000,...,0.019098,0.020873,84.337500,-3.804615,0.408308,-8.395385,-5.596308,0.380615,-10.599385,62.357641
2,20120101,경기,1,4,1024.840000,1027.080000,1021.920000,-3.720000,0.720000,-8.180000,...,0.011926,0.030821,86.678511,-4.037975,0.314937,-8.975696,-5.211899,0.261772,-12.079747,65.194203
3,20120101,경기,2,5,1024.840000,1027.080000,1021.920000,-3.720000,0.720000,-8.180000,...,0.011926,0.030821,86.678511,-4.037975,0.314937,-8.975696,-5.211899,0.261772,-12.079747,65.194203
4,20120101,경남,1,2,1022.771429,1024.850000,1019.528571,1.157143,6.650000,-3.250000,...,0.021260,0.017499,62.025845,0.520455,5.850000,-4.050000,-2.434091,5.850000,-7.215909,55.743434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49669,20151231,제주,2,0,1030.200000,1033.700000,1026.500000,8.325000,10.750000,6.075000,...,0.034771,0.009910,74.854167,5.770588,8.088235,3.288235,5.300000,7.717647,2.029412,59.915541
49670,20151231,충남,1,0,1031.133333,1034.066667,1027.050000,1.200000,6.000000,-2.783333,...,0.017996,0.019843,53.587500,1.973333,6.040000,-1.400000,0.590000,6.000000,-2.333333,74.725225
49671,20151231,충남,2,0,1031.133333,1034.066667,1027.050000,1.200000,6.000000,-2.783333,...,0.017996,0.019843,53.587500,1.973333,6.040000,-1.400000,0.590000,6.000000,-2.333333,74.725225
49672,20151231,충북,1,1,1030.980000,1034.540000,1027.260000,0.480000,4.040000,-3.680000,...,0.009787,0.028711,64.007968,0.308696,4.504348,-4.423913,-0.367391,4.504348,-6.645652,70.658784


In [72]:
merged_7.to_csv('/home/jupyter/climate/data/final_train.csv')

## 모델평가용 데이터 병합

In [10]:
hospital = pd.read_csv('/home/jupyter/climate/data/back_testset.csv', header = 0)
sfc_prsr = pd.read_csv('/home/jupyter/climate/data/final_sfc_prsr_0723.csv', header = 0)
sfc_td = pd.read_csv('/home/jupyter/climate/data/final_sfc_td_0723.csv', header = 0)
sfc_ta = pd.read_csv('/home/jupyter/climate/data/final_sfc_ta_0723.csv', header = 0)
ydst = pd.read_csv('/home/jupyter/climate/data/final_ydst_pm10_ver1_0723.csv', header = 0)
aws_ta = pd.read_csv('/home/jupyter/climate/data/final_aws_ta_0713.csv', header = 0)
fore_rhm = pd.read_csv('/home/jupyter/climate/data/final_fore_rhm_0801_2313.csv', header = 0)
hospital = hospital.rename(columns={'back_hospital.area' : 'add'})
hospital = hospital.rename(columns={'back_hospital.yyyymmdd' : 'yyyymmdd'})
hospital = hospital.rename(columns={'back_hospital.sex' : 'sex'})
hospital = hospital.rename(columns={'back_hospital.frequency' : 'frequency'})
ydst = ydst.rename(columns={'area' : 'add'})

hospital = hospital.rename(columns={'area' : 'add'})
ydst = ydst.rename(columns={'area' : 'add'})

In [11]:
prsr_sejong = sfc_prsr[sfc_prsr['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_prsr = sfc_prsr.append(prsr_sejong)

ta_sejong = sfc_ta[sfc_ta['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_ta = sfc_ta.append(ta_sejong)

td_sejong = sfc_td[sfc_td['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_td = sfc_td.append(td_sejong)

In [12]:
sfc_prsr_grouped = sfc_prsr.groupby(['yyyymmdd','add']).mean()
sfc_ta_grouped = sfc_ta.groupby(['yyyymmdd','add']).mean()
sfc_td_grouped = sfc_td.groupby(['yyyymmdd','add']).mean()
ydst_grouped = ydst.groupby(['yyyymmdd','add']).mean()
aws_ta_grouped = aws_ta.groupby(['yyyymmdd','add']).mean()
fore_rhm_grouped = fore_rhm.groupby(['yyyymmdd','add']).mean()

### 제공된 테스트 데이터의 날짜 형식 변환

In [76]:
hospital['yyyymmdd']

0        2016-01-01
1        2016-01-01
2        2016-01-01
3        2016-01-01
4        2016-01-01
            ...    
12439    2016-12-31
12440    2016-12-31
12441    2016-12-31
12442    2016-12-31
12443    2016-12-31
Name: yyyymmdd, Length: 12444, dtype: object

In [13]:
hospital['yyyymmdd'] = pd.to_datetime(hospital['yyyymmdd'], format='%Y-%m-%d')
hospital['yyyymmdd'] = hospital['yyyymmdd'].dt.strftime('%Y%m%d')
hospital['yyyymmdd'] = hospital['yyyymmdd'].astype(int)

In [14]:
merged_1 = pd.merge(hospital, sfc_prsr_grouped, on = ['add','yyyymmdd'])
merged_2 = pd.merge(merged_1, sfc_ta_grouped, on = ['add','yyyymmdd'])
merged_3 = pd.merge(merged_2, sfc_ta_grouped, on = ['add','yyyymmdd'])
merged_4 = pd.merge(merged_3, sfc_td_grouped, on = ['add','yyyymmdd'])
merged_5 = pd.merge(merged_4, ydst_grouped, on = ['add','yyyymmdd'])
merged_6 = pd.merge(merged_5, aws_ta_grouped, on = ['add','yyyymmdd'])
merged_7 = pd.merge(merged_6, fore_rhm_grouped, on = ['add','yyyymmdd'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'lat_x', 'long_x', 'stn_id_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'lat_x', 'long_x', 'stn_id_x'} in the result is deprecated and will raise a MergeError in a future version.
  import sys


In [79]:
merged_7.columns

Index(['yyyymmdd', 'add', 'sex', 'frequency', 'stn_id_x', 'avg_ps', 'max_ps',
       'min_ps', 'lat_x', 'long_x', 'stn_id_y', 'avg_ta_x', 'max_ta_x',
       'min_ta_x', 'lat_y', 'long_y', 'stn_id_x', 'avg_ta_y', 'max_ta_y',
       'min_ta_y', 'lat_x', 'long_x', 'stn_id_y', 'avg_td', 'max_td',
       'max_td_hrmt', 'min_td', 'min_td_hrmt', 'lat_y', 'long_y', 'so2', 'co',
       'o3', 'no2', 'pm10', 'isnull', 'Unnamed: 0', 'stn_id_x', 'avg_ta',
       'max_ta', 'min_ta', 'min_mrng_ta', 'max_dytm_ta', 'min_nght_ta',
       'lat_x', 'long_x', 'value', 'stn_id_y', 'long_y', 'lat_y'],
      dtype='object')

In [16]:
merged_7.drop(['stn_id_x', 'lat_x', 'long_x', 'isnull', 'stn_id_y', 'lat_y', 'long_y', 'Unnamed: 0', 'min_mrng_ta', 'max_dytm_ta', 'min_nght_ta'], axis=1, inplace=True)

### 데이터 확인 및 저장 저장

In [17]:
merged_7

,yyyymmdd,add,sex,frequency,avg_ps,max_ps,min_ps,avg_ta_x,max_ta_x,min_ta_x,...,min_td_hrmt,so2,co,o3,no2,pm10,avg_ta,max_ta,min_ta,value
0,20160101,강원,1,NaN,1029.855714,1033.395714,1025.952857,0.388571,4.777143,-4.795714,...,757.142857,0.004080,0.822500,0.019187,0.019066,55.840336,-0.970462,3.581538,-6.461846,58.196911
1,20160101,강원,2,NaN,1029.855714,1033.395714,1025.952857,0.388571,4.777143,-4.795714,...,757.142857,0.004080,0.822500,0.019187,0.019066,55.840336,-0.970462,3.581538,-6.461846,58.196911
2,20160101,경기,1,NaN,1031.200000,1034.340000,1027.280000,-0.300000,4.540000,-5.940000,...,500.000000,0.005356,0.935390,0.008361,0.037651,76.451042,-0.053629,4.611392,-5.794051,70.844595
3,20160101,경기,2,NaN,1031.200000,1034.340000,1027.280000,-0.300000,4.540000,-5.940000,...,500.000000,0.005356,0.935390,0.008361,0.037651,76.451042,-0.053629,4.611392,-5.794051,70.844595
4,20160101,경남,1,NaN,1031.042857,1034.064286,1028.457143,1.764286,9.057143,-4.264286,...,800.000000,0.004431,0.666919,0.016600,0.024610,55.407609,1.026818,8.636818,-4.885000,52.722973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,20161231,제주,2,NaN,1030.175000,1031.850000,1029.100000,6.700000,10.025000,3.750000,...,575.000000,0.001337,0.190526,0.029063,0.012552,23.880597,4.682353,8.264706,1.117647,62.488739
12440,20161231,충남,1,NaN,1030.250000,1031.666667,1028.683333,1.316667,6.233333,-2.483333,...,116.666667,0.002730,0.631250,0.014071,0.022291,42.137500,2.563333,6.806667,-0.843333,68.924550
12441,20161231,충남,2,NaN,1030.250000,1031.666667,1028.683333,1.316667,6.233333,-2.483333,...,116.666667,0.002730,0.631250,0.014071,0.022291,42.137500,2.563333,6.806667,-0.843333,68.924550
12442,20161231,충북,1,NaN,1030.740000,1032.320000,1029.000000,-0.740000,3.700000,-5.440000,...,380.000000,0.004405,0.905903,0.005111,0.026913,51.435540,-0.911304,4.100000,-5.458261,66.781532


In [82]:
merged_7.to_csv('/home/jupyter/climate/data/final_test.csv')